# WebScraping NewEgg.com

In [21]:
# Import dependencies
from bs4 import BeautifulSoup as soup
import requests
import pymongo
import numpy as np
import pandas as pd
from splinter import Browser
from selenium import webdriver
import time
import re

In [2]:
# Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [3]:
# Define database
# db = client.newegg_laptops_db
# # Define collection
# collection = db.apple

In [4]:
# Setup Splinter inputs

executable_path = {'executable_path': './chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# set up url that you want to scrape
#url = 'https://www.newegg.com/p/pl?N=100006740&page=1&order=RELEASE'
url = 'https://www.newegg.com/'

In [6]:
# Query Input

#query_input = input("What products would you like to scrape?")
query_input = "Laptops"

In [7]:
# instruct web browser to visit the url
browser.visit(url)

In [8]:
# Find the search bar, and type in "laptops"
search_box = browser.find_by_id("SearchBox2020").type(query_input)

In [9]:
# Click the search button! - pretty hard to figure out but did it!

#submit = browser.find_by_name("fas fa-search").first.click()
#submit = browser.find_link_by_text("fas fa-search").first.click()

####### This works!
# First looked for all css style sheets in the head
# Looked at the names of each css file
# and dug up the exact location of the css folder
# look for a similar class. narrow it down.
# check if it existed
# then used the last method
#browser.find_by_css(".fa-search").first.has_class('fa-search')
submit = browser.find_by_css(".fa-search").first.click()

In [10]:
# Click on the first element which is laptops / notebooks on the side menu far target by class
browser.find_by_css(".filter-box-label").first.click()

In [ ]:
# This finds the drop down menu for "Sort By" -- will skip this
#browser.find_by_id("Order_top").first.click()

In [ ]:
# page_soup = soup(browser.html, 'lxml')
# page_soup

In [11]:
# Click on "Sold By: Newegg" - later add options for each type (later code by xpath for each option)

browser.find_by_css(".form-radiobox-title").first.click()

In [12]:
current_url = browser.url

In [13]:
response = requests.get(current_url)
response

<Response [200]>

In [14]:
current_page_soup = soup(response.text, 'lxml')
#current_page_soup

In [23]:
# Extract total page numbers from results

results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()

In [26]:
# Find and extract total pages

total_results_pages = int(re.split("/", results_pages)[1]) + 1

In [29]:
for turn_page in range(1, total_results_pages):
    browser.find_by_xpath('//*[@id="bodyArea"]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()
    time.sleep(2)

AttributeError: 'ElementList' object has no attribute 'click'

In [ ]:
# This is "NEXT PAGE BUTTON CLICK". Now need to create a loop for the length of the total respective category

# this step or function would come after webscraping one page - this will definitely need to go in a loop

# while page_counter > total_results pages: keep scraping.

browser.find_by_xpath('//*[@id="bodyArea"]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

In [ ]:

#browser.find_by_text("2").click()
#browser.find_by_css(".list-tool-pagination-text").first.has_class('list-tool-pagination-text')#.first.click()
#browser.find_by_css(".btn-group-cell").has_class('btn-group-cell')

browser.find_by_css(".btn-group-cell").click()

In [ ]:
# Build a page flip loop using selenium
# get the page counter down

## Page Scraper And Exports Out to CSV. Later can figure out how to get it out to PyMongo / Mongo Db.

In [ ]:
# instruct webdriver this is the html page to scrape
# splinter .html will set the 'source of the page content' 
html = browser.html
# Use beautiful soup to parse the text using html.parser
page_soup = soup(html, 'html.parser')

# Capture all containers on the page to set up for looping thru later
containers = page_soup.find_all("div", class_="item-container") 



In [ ]:
# Build the loop to extract: brand, Title of the product, price-dollar, price-cents,
# shipping


def newegg_page_scraper(containers):

    laptop_brands = []
    laptop_models = []
    laptop_prices = []
    laptop_shipping = []

    for con in containers:
        page_counter = 0
        page_counter += 1
        brand_name = con.find_all('a', class_="item-brand")[0].img["title"]
        laptop_brands.append(brand_name)
        #print(brand_name)

        prd_title = con.find_all('a', class_="item-title")[0].text
        laptop_models.append(prd_title)

        price = con.find_all('li', class_="price-current")[0].text.split()[0]
        laptop_prices.append(price)

        shipping = contain.find_all('li', class_='price-ship')[0].text.strip()
        laptop_shipping.append(shipping)

    df = pd.DataFrame({
    'brand': laptop_brands,
    'model_listing': laptop_models,
    'price': laptop_prices,
    'shipping': laptop_shipping
    })

    df.to_csv(f'./output/newgg_laptop_page_{page_counter}.csv')
    
    return f"Completed scraping laptop section page number: {page_counter} . "
    
newegg_page_scraper(containers)

In [ ]:
df